In [ ]:
systemctl启动服务

In [ ]:
1.镜像下载
[root@docker-server ~]# docker pull daocloud.io/library/centos:7

In [ ]:
2.systemed整合
因为 systemd 要求 CAPSYSADMIN 权限，从而得到了读取到宿主机 cgroup 的能力，CentOS7 中已经用 fakesystemd 代替了 systemd 。
但是使用systemd，可用参考下面的 Dockerfile。这个Dockerfile删除fakesystemd 并安装了 systemd。

In [ ]:
[root@docker-server ~]# mkdir test
[root@docker-server ~]# cd test/
[root@docker-server test]# vim Dockerfile
FROM daocloud.io/library/centos:7
MAINTAINER "soso"  soso@qq.com
ENV container docker
RUN yum -y swap -- remove fakesystemd -- install systemd systemd-libs
RUN yum -y update; yum clean all; \
(cd /lib/systemd/system/sysinit.target.wants/; for i in *; do [ $i == systemd-tmpfiles-setup.service ] || rm -f $i; done); \
rm -f /lib/systemd/system/multi-user.target.wants/*;\
rm -f /etc/systemd/system/*.wants/*;\
rm -f /lib/systemd/system/local-fs.target.wants/*; \
rm -f /lib/systemd/system/sockets.target.wants/*udev*; \
rm -f /lib/systemd/system/sockets.target.wants/*initctl*; \
rm -f /lib/systemd/system/basic.target.wants/*;\
rm -f /lib/systemd/system/anaconda.target.wants/*;
VOLUME [ "/sys/fs/cgroup" ]   #指定逻辑卷
CMD ["/usr/sbin/init"]     #代表的是把上面的服务拉起来

In [ ]:
Cgroups（Control Groups）是Linux内核中用于对进程进行资源限制、优先级分配和审计的重要机制，宿主机的cgroup在docker中扮演着关键的角色：
#资源限制
-CPU限制：在宿主机上，cgroup可以为docker容器分配一定比例的CPU资源。例如，一个宿主机的CPU核心数为4，在docker中可以通过cgroup设置某个容器只能使用其中的两个核心或者一定比例的CPU时间。
-内存限制：cgroup允许限制容器使用的最大内存。如果容器尝试使用的内存超出分配的配额，宿主机的cgroup会阻止容器分配更多的内存，防止因某个容器内存使用失控而导致宿主机或其他容器的内存不足问题。
-磁盘I/O限制：可以对容器的磁盘读写速度和IOPS（每秒I/O操作数）进行限制。
#资源隔离
-隔离CPU和内存：cgroup能够使容器的进程组与宿主机及其他容器的进程组在CPU和内存使用上相互隔离。
-隔离文件系统：虽然cgroup本身主要用于资源限制而不直接处理文件系统隔离，但它与Docker的文件系统隔离（如通过UnionFS或overlay2存储驱动程序实现的文件系统分层和隔离）共同作用，确保容器在文件系统层面的行为不会影响宿主机或其他容器。
#资源审计：资源消耗统计，审计资源分配
#分布式系统中资源的全局管理

In [ ]:
为了使用上面那样包含systemd的容器，需要创建类似下面的dockerfile

In [ ]:
[root@docker-server test]# mkdir http
[root@docker-server test]# cd http/
[root@docker-server http]# vim Dockerfile
FROM local/c7-systemd
RUN yum -y install httpd; yum clean all; systemctl enable httpd.service
EXPOSE 80
CMD ["/usr/sbin/init"]

或者
[root@docker-server http]# cat Dockerfile 
FROM local/c7-systemd
RUN rm -rf /etc/yum.repos.d/*
ADD Centos-7.repo /etc/yum.repos.d/
RUN yum -y install httpd; yum clean all; systemctl enable httpd
EXPOSE 80
CMD ["/usr/sbin/init"]

In [ ]:
#构建镜像
docker build -t local/c7-systemd-httpd .

In [ ]:
3.运行包含 systemd 的应用容器
为了运行一个包含 systemd 的容器，需要使用--privileged选项， 并且挂载主机的 cgroups 文件夹。 
下面是运行包含 systemd 的 httpd 容器的示例命令：
[root@docker-server http]# docker run --privileged -tid -v /sys/fs/cgroup:/sys/fs/cgroup:ro -p 80:80 local/c7-systemd-httpd
--privileged:授权提权。让容器内的root用户拥有真正root权限(有些权限是没有的)
-ro代表的是只读
-rw代表的是读写
如果不加会运行在前台(没有用-d)，可以用ctrl+p+q放到后台去

In [ ]:
4.安装openssh-server的例子
[root@docker-server http]# cd ..
[root@docker-server test]# mkdir ssh
[root@docker-server test]# cd ssh/
[root@docker-server ssh]# vim Dockerfile
FROM local/c7-systemd
RUN yum -y install openssh-server; yum clean all; systemctl enable sshd.service
RUN echo 123456 | passwd --stdin root
EXPOSE 22
CMD ["/usr/sbin/init"]
[root@docker-server ssh]# docker build -t local/c7-systemd-sshd .
[root@docker-server ssh]# docker run --privileged -tid -v /sys/fs/cgroup:/sys/fs/cgroup:ro -p 2222:22 local/c7-systemd-sshd
[root@docker-server ssh]# ssh 192.168.246.141 -p 2222
[root@ce1af52a6f6c ~]#